Em primeira instãncia , realizamos a instalação das bibliotecas:

In [1]:
!pip install fasttext-wheel

In [2]:
import fasttext

In [3]:
import csv

In [4]:
!pip install pandas
!pip install numpy

In [5]:
import pandas as pd
import numpy as np

import gzip
import json

Abertura do arquivo:

In [6]:
file_path = "C:\\Caio\\Owl\\aotm2011_playlists.json.gz"
with gzip.open(file_path, 'rt', encoding='utf-8') as f:
    data = json.load(f)

Nosso DataSet das playlists em formato de DataFrame, e sem alterações prévias:

In [7]:
playlists = pd.read_json(file_path)
playlists

,category,filtered_lists,playlist,timestamp,mix_id,user
0,Mixed Genre,"[[SOFDPDC12A58A7D198], [SOPIEQP12A8C13F268, SO...","[[[peter murphy, marlene dietrich's favourite ...",2005-03-27 10:53:00,89567,"{'mixes_posted': '23', 'member_since': '2004-0..."
1,Theme,"[[SOQOVCB12A58A7E83B], [SOGKLYN12AB017F155, SO...","[[[The Cure, Edge Of the Deep Green Sea], None...",2008-01-11 21:45:00,117124,"{'mixes_posted': '7', 'member_since': '2006-11..."
2,Theme,[[SOFIQBR12A6D4F78C6]],"[[[Peter Gabriel, The Feeling Begins], None], ...",2005-10-31 10:15:00,97418,"{'mixes_posted': '93', 'member_since': '2003-1..."
3,Theme,"[[SORJUFZ12A67ADAF25, SOEELNN12AB018AC8D], [SO...","[[[The Animals, San Francisco Nights], None], ...",2002-03-20 17:59:00,23876,"{'mixes_posted': '8', 'member_since': '2002-03..."
4,Theme,"[[SOOUYTP12B0B8094B6, SOAJODS12A8C138A23], [SO...","[[[Sparklehorse, It's a Wonderful Life], SOOUY...",2004-03-23 03:43:00,70694,"{'mixes_posted': '60', 'member_since': '2004-0..."
...,...,...,...,...,...,...
101338,Hip Hop,"[[SODLSVK12A8C13EEE1], [SOEOMEJ12A8AE46299], [...","[[[R. Kelly, Burn It Up (Remix feat. Daddy Yan...",2006-04-03 14:54:00,102561,"{'mixes_posted': '24', 'member_since': '2003-0..."
101339,Mixed Genre,"[[SONYEIJ12A6D4FA32D, SOSGUDA12A6D4FA2FB], [SO...","[[[Pachelbel, Grave], None], [[Prefuse73, Afte...",2004-03-03 15:05:00,69379,"{'mixes_posted': '1', 'member_since': '2004-03..."
101340,Single Artist,[[SOYOFST12D0219B4BA]],"[[[Queen, Bohemian Rhapsody], SOYOFST12D0219B4...",2001-12-18 00:01:00,18933,"{'mixes_posted': '23', 'member_since': '2001-1..."
101341,Theme,"[[SOKAWYU12A6D4F6769, SOXMBFV12A8C13B8DD], [SO...","[[[birthday party, big-jesus-trash-can], None]...",2006-01-28 10:16:00,100251,"{'mixes_posted': '40', 'member_since': '2002-0..."


Pegaremos apenas as colunas que nos parecem interessantes:

In [8]:
playlists = playlists.drop(columns = ['user', 'mix_id', 'filtered_lists'])

In [9]:
playlists

,category,playlist,timestamp
0,Mixed Genre,"[[[peter murphy, marlene dietrich's favourite ...",2005-03-27 10:53:00
1,Theme,"[[[The Cure, Edge Of the Deep Green Sea], None...",2008-01-11 21:45:00
2,Theme,"[[[Peter Gabriel, The Feeling Begins], None], ...",2005-10-31 10:15:00
3,Theme,"[[[The Animals, San Francisco Nights], None], ...",2002-03-20 17:59:00
4,Theme,"[[[Sparklehorse, It's a Wonderful Life], SOOUY...",2004-03-23 03:43:00
...,...,...,...
101338,Hip Hop,"[[[R. Kelly, Burn It Up (Remix feat. Daddy Yan...",2006-04-03 14:54:00
101339,Mixed Genre,"[[[Pachelbel, Grave], None], [[Prefuse73, Afte...",2004-03-03 15:05:00
101340,Single Artist,"[[[Queen, Bohemian Rhapsody], SOYOFST12D0219B4...",2001-12-18 00:01:00
101341,Theme,"[[[birthday party, big-jesus-trash-can], None]...",2006-01-28 10:16:00


Apartir de agora, iremos testar um modelo para remover alguns caracteres indesejados da coluna playlist, como os colchetes por exemplo.

In [10]:
song_list = (playlists.iloc[1]['playlist'])
cleaned_list = [f"{band}::{song}" for (band, song), _ in song_list]
cleaned_list = ';'.join(cleaned_list)
print(cleaned_list)

The Cure::Edge Of the Deep Green Sea;Jeff Buckley::Nightmares By the Sea;Travis::Driftwood;Johnny Cash::Big River;G.Love::Fishing Song;REM::Nightswimming;The Doves::Caught By the River;Sugar::A Good Idea;Badly Drawn Boy::River, Sea, Ocean;Apples In Stereo::Stream Running Over;Frank Black & The Catholics::St. Francis Damn Disaster;The Tragically Hip::New Orleans Is Sinking;Tom Waits::Rain;Morphine::Wishing Well;They Might Be Giants::Cowtown;Talking Heads::Take Me To the River;Mike Doughty::Down On the River By the Sugar Plant;Porno For Pyros::Tahitian Moon;James::Sometimes


Removemos os espaços também

In [11]:
cleaned_list = cleaned_list.replace(' ', '')
cleaned_list

'TheCure::EdgeOftheDeepGreenSea;JeffBuckley::NightmaresBytheSea;Travis::Driftwood;JohnnyCash::BigRiver;G.Love::FishingSong;REM::Nightswimming;TheDoves::CaughtBytheRiver;Sugar::AGoodIdea;BadlyDrawnBoy::River,Sea,Ocean;ApplesInStereo::StreamRunningOver;FrankBlack&TheCatholics::St.FrancisDamnDisaster;TheTragicallyHip::NewOrleansIsSinking;TomWaits::Rain;Morphine::WishingWell;TheyMightBeGiants::Cowtown;TalkingHeads::TakeMeTotheRiver;MikeDoughty::DownOntheRiverBytheSugarPlant;PornoForPyros::TahitianMoon;James::Sometimes'

Fazemos então uma função que agrupa os passos anteriores: remove os colchetes, retira os espaços e faz devidas alterações na capitalização da palavra(MAIÚSCULA Ou minúscula)

In [12]:
def limpador(song_list):
    def capitalize_words(text):
        return ''.join(word.capitalize() for word in text.split())

    def remove_special_chars(text):
        special_chars = "!@#$%^&*()_+={}[]|\:;\"'<>?,./"
        for char in special_chars:
            text = text.replace(char, "")
        return text

    cleaned_list = [(capitalize_words(band), capitalize_words(song)) for (band, song), _ in song_list]
    cleaned_list = [f"{remove_special_chars(band)}::{remove_special_chars(song)}" for (band, song) in cleaned_list]
    cleaned_string = ';'.join(cleaned_list)
    return cleaned_string


Criamos então uma nova coluna, com todos os termos da coluna 'playlist' e aplicamos a função anterior em todas as linhas


In [13]:
playlist_limpa = playlists.get('playlist').apply(limpador)

In [14]:
playlists = playlists.assign(
    playlists_limpas = playlist_limpa
)
playlists

,category,playlist,timestamp,playlists_limpas
0,Mixed Genre,"[[[peter murphy, marlene dietrich's favourite ...",2005-03-27 10:53:00,PeterMurphy::MarleneDietrichsFavouritePoem;The...
1,Theme,"[[[The Cure, Edge Of the Deep Green Sea], None...",2008-01-11 21:45:00,TheCure::EdgeOfTheDeepGreenSea;JeffBuckley::Ni...
2,Theme,"[[[Peter Gabriel, The Feeling Begins], None], ...",2005-10-31 10:15:00,PeterGabriel::TheFeelingBegins;OfraHazaAndIggy...
3,Theme,"[[[The Animals, San Francisco Nights], None], ...",2002-03-20 17:59:00,TheAnimals::SanFranciscoNights;TheZombies::Tim...
4,Theme,"[[[Sparklehorse, It's a Wonderful Life], SOOUY...",2004-03-23 03:43:00,Sparklehorse::ItsAWonderfulLife;Whiskeytown::I...
...,...,...,...,...
101338,Hip Hop,"[[[R. Kelly, Burn It Up (Remix feat. Daddy Yan...",2006-04-03 14:54:00,RKelly::BurnItUpremixFeatDaddyYankee;50Cent::C...
101339,Mixed Genre,"[[[Pachelbel, Grave], None], [[Prefuse73, Afte...",2004-03-03 15:05:00,Pachelbel::Grave;Prefuse73::AfternoonLoveIn;Ni...
101340,Single Artist,"[[[Queen, Bohemian Rhapsody], SOYOFST12D0219B4...",2001-12-18 00:01:00,Queen::BohemianRhapsody;::TieYourMotherDown;::...
101341,Theme,"[[[birthday party, big-jesus-trash-can], None]...",2006-01-28 10:16:00,BirthdayParty::Big-jesus-trash-can;XmalDeutsch...


Criamos uma nova coluna, chamada 'new_category' que será o equivalente a 'category', porém, sem espaçamentos entre os termos. Mais adiante ficará mais claro o porquê disso.

In [15]:
import re

In [16]:
def processador(coluna):
    nova_coluna = re.sub(' ','',coluna)
    return nova_coluna

In [17]:
playlists['new_category'] = playlists['category'].map(processador)
playlists

,category,playlist,timestamp,playlists_limpas,new_category
0,Mixed Genre,"[[[peter murphy, marlene dietrich's favourite ...",2005-03-27 10:53:00,PeterMurphy::MarleneDietrichsFavouritePoem;The...,MixedGenre
1,Theme,"[[[The Cure, Edge Of the Deep Green Sea], None...",2008-01-11 21:45:00,TheCure::EdgeOfTheDeepGreenSea;JeffBuckley::Ni...,Theme
2,Theme,"[[[Peter Gabriel, The Feeling Begins], None], ...",2005-10-31 10:15:00,PeterGabriel::TheFeelingBegins;OfraHazaAndIggy...,Theme
3,Theme,"[[[The Animals, San Francisco Nights], None], ...",2002-03-20 17:59:00,TheAnimals::SanFranciscoNights;TheZombies::Tim...,Theme
4,Theme,"[[[Sparklehorse, It's a Wonderful Life], SOOUY...",2004-03-23 03:43:00,Sparklehorse::ItsAWonderfulLife;Whiskeytown::I...,Theme
...,...,...,...,...,...
101338,Hip Hop,"[[[R. Kelly, Burn It Up (Remix feat. Daddy Yan...",2006-04-03 14:54:00,RKelly::BurnItUpremixFeatDaddyYankee;50Cent::C...,HipHop
101339,Mixed Genre,"[[[Pachelbel, Grave], None], [[Prefuse73, Afte...",2004-03-03 15:05:00,Pachelbel::Grave;Prefuse73::AfternoonLoveIn;Ni...,MixedGenre
101340,Single Artist,"[[[Queen, Bohemian Rhapsody], SOYOFST12D0219B4...",2001-12-18 00:01:00,Queen::BohemianRhapsody;::TieYourMotherDown;::...,SingleArtist
101341,Theme,"[[[birthday party, big-jesus-trash-can], None]...",2006-01-28 10:16:00,BirthdayParty::Big-jesus-trash-can;XmalDeutsch...,Theme


Novamente, organizamos as colunas e filtramos apenas a que iremos usar:

In [18]:
playlists = playlists[['new_category','playlists_limpas', 'timestamp']]
playlists

,new_category,playlists_limpas,timestamp
0,MixedGenre,PeterMurphy::MarleneDietrichsFavouritePoem;The...,2005-03-27 10:53:00
1,Theme,TheCure::EdgeOfTheDeepGreenSea;JeffBuckley::Ni...,2008-01-11 21:45:00
2,Theme,PeterGabriel::TheFeelingBegins;OfraHazaAndIggy...,2005-10-31 10:15:00
3,Theme,TheAnimals::SanFranciscoNights;TheZombies::Tim...,2002-03-20 17:59:00
4,Theme,Sparklehorse::ItsAWonderfulLife;Whiskeytown::I...,2004-03-23 03:43:00
...,...,...,...
101338,HipHop,RKelly::BurnItUpremixFeatDaddyYankee;50Cent::C...,2006-04-03 14:54:00
101339,MixedGenre,Pachelbel::Grave;Prefuse73::AfternoonLoveIn;Ni...,2004-03-03 15:05:00
101340,SingleArtist,Queen::BohemianRhapsody;::TieYourMotherDown;::...,2001-12-18 00:01:00
101341,Theme,BirthdayParty::Big-jesus-trash-can;XmalDeutsch...,2006-01-28 10:16:00


Aqui , para deixar claro algumas informações, foi usado groupby para sabermos a relação da quantidade de playlist(já limpas) com as categorias, o quê inclusive nos leva a observar e descobrir quais são as categorias do DataSet

In [19]:
playlists.groupby('new_category').count()

,playlists_limpas,timestamp
new_category,,
AlternatingDJ,5053,5053
Ambient,306,306
AvantGarde,110,110
Blues,197,197
BreakUp,1115,1115
Classical,73,73
Country,484,484
Cover,657,657
Dance/House,706,706


Tomamos então uma ação importante: 
Criaremos uma nova coluna chamada 'playlist_limpas_indicadas', e junto disso , adicionaremos o termo "__ label __" seguido de um espaço e o nome da categoria pertecente a essa playlist. Isso será muito importante para algumas funções usando a bilioteca 'fasttext'.

In [20]:
playlists['playlists_limpas_indicadas'] = "__label__"+playlists['new_category']+" "+playlists['playlists_limpas']
playlists = playlists[['new_category','playlists_limpas_indicadas', 'timestamp']]
playlists

C:\Users\ccaio\AppData\Local\Temp\ipykernel_13472\3116498201.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  playlists['playlists_limpas_indicadas'] = "__label__"+playlists['new_category']+" "+playlists['playlists_limpas']


,new_category,playlists_limpas_indicadas,timestamp
0,MixedGenre,__label__MixedGenre PeterMurphy::MarleneDietri...,2005-03-27 10:53:00
1,Theme,__label__Theme TheCure::EdgeOfTheDeepGreenSea;...,2008-01-11 21:45:00
2,Theme,__label__Theme PeterGabriel::TheFeelingBegins;...,2005-10-31 10:15:00
3,Theme,__label__Theme TheAnimals::SanFranciscoNights;...,2002-03-20 17:59:00
4,Theme,__label__Theme Sparklehorse::ItsAWonderfulLife...,2004-03-23 03:43:00
...,...,...,...
101338,HipHop,__label__HipHop RKelly::BurnItUpremixFeatDaddy...,2006-04-03 14:54:00
101339,MixedGenre,__label__MixedGenre Pachelbel::Grave;Prefuse73...,2004-03-03 15:05:00
101340,SingleArtist,__label__SingleArtist Queen::BohemianRhapsody;...,2001-12-18 00:01:00
101341,Theme,__label__Theme BirthdayParty::Big-jesus-trash-...,2006-01-28 10:16:00


Baixaremos  e importaremos uma importante biblioteca, que nos auxiliará na construção de arquivos vindo do nosso DataSet

In [21]:
!pip install scikit-learn

In [22]:
from sklearn.model_selection import train_test_split


Definiremos então tamanhos e proporções para os nossos arquivos de teste

In [23]:
train,test = train_test_split(playlists,test_size = 0.1)

In [24]:
display(train.shape)
display(test.shape)

(91208, 3)

(10135, 3)

Nesse momento, usando os seguintes passos, atribuiremos a um arquivo (esse passo pode ser executado criando um novo arquivo) linhas que contém, uma por uma, as informações de cada playlist. Atenta-se ao fato, de que , caso seja rodado em outro dispositivo, realizar as devidas alterações quanto ao nome do arquivo

In [25]:
train.to_csv(r'C:\\Caio\\Owl\\Train and test\\new_traine.csv',columns=['playlists_limpas_indicadas'],index=False,header=False)
test.to_csv(r'C:\\Caio\\Owl\\Train and test\\new_teste.csv',columns=['playlists_limpas_indicadas'],index=False,header=False)

Treinaremos então um modelo , usando funções provinientes da biblioteca 'fasttext' e atribuindo a variável 'modelo'

In [26]:
modelo = fasttext.train_supervised(input="C:\\Caio\\Owl\\Train and test\\new_traine.csv",lr=0.1,epoch=20,wordNgrams=2,dim=100,loss='softmax')

Usaremos o nosso o nosso arquivo de teste nesse momento

In [27]:
modelo.test(r"C:\\Caio\\Owl\\Train and test\\new_teste.csv")

(10135, 0.45268870251603355, 0.45268870251603355)

Aqui usaremos a função .predict da biblioteca fasttext para prever a classe de um texto fornecido, com base no modelo treinado.

In [28]:
modelo.predict("TheBeatles")

(('__label__MixedGenre',), array([0.56748873]))

Ao executar o teste com outro termo qualquer, observamos que, o resultado é o mesmo quanto a classe. Foi feito ínumeros teste e sempre constaram o mesmo resultado.

In [29]:
modelo.predict("Gabriel Lindão")

(('__label__MixedGenre',), array([0.408986]))

Dado isso, observa-se que uma possível causa para isso é o fato de haver um número abrupdamente alto de playlists ligada a categoria MixedGenre

In [30]:
playlists.groupby('new_category').count()

,playlists_limpas_indicadas,timestamp
new_category,,
AlternatingDJ,5053,5053
Ambient,306,306
AvantGarde,110,110
Blues,197,197
BreakUp,1115,1115
Classical,73,73
Country,484,484
Cover,657,657
Dance/House,706,706


Removeremos os elementos relacionados a essa categoria para prosseguirmos. Essa decisão foi tomada com base no fato dessa categoria ser consideravelmente generalizada.

In [31]:
playlists = playlists[playlists['new_category'] != 'MixedGenre']
playlists.groupby('new_category').count()


,playlists_limpas_indicadas,timestamp
new_category,,
AlternatingDJ,5053,5053
Ambient,306,306
AvantGarde,110,110
Blues,197,197
BreakUp,1115,1115
Classical,73,73
Country,484,484
Cover,657,657
Dance/House,706,706


Dado isso , efetuaremos o mesmo procedimento de antes da remoção, dessa vez, criaremos um novo arquivo de teste e treino, para evitar quaisquer erros.

In [32]:
train,test = train_test_split(playlists,test_size = 0.1)

In [33]:
display(train.shape)
display(test.shape)

(49734, 3)

(5526, 3)

In [34]:
train.to_csv(r'C:\\Caio\\Owl\\Train and test\\new_new_traine.csv',columns=['playlists_limpas_indicadas'],index=False,header=False)
test.to_csv(r'C:\\Caio\\Owl\\Train and test\\new_new_teste.csv',columns=['playlists_limpas_indicadas'],index=False,header=False)

In [35]:
novo_modelo = fasttext.train_supervised(
    input=r"C:\\Caio\\Owl\\Train and test\\new_new_traine.csv"
)

In [36]:
novo_modelo.test(r"C:\\Caio\\Owl\\Train and test\\new_new_teste.csv")
result = novo_modelo.test("C:\\Caio\\Owl\\Train and test\\new_new_teste.csv")
print(f"Precisão: {result[1]}")
print(f"Recall: {result[2]}")

Precisão: 0.27651103872602245
Recall: 0.27651103872602245


In [37]:
novo_modelo.predict('TheSmiths')

(('__label__Theme',), array([0.27486861]))

**LEMBRAR DE EXCLUIR ESSA CÉLULA**

Usei como base o vídeo de um indiano estranho:https://youtu.be/ahYRKzYYCIw?si=zj7zKkUt6crDNHD5
O principal erro até entao , pode estar no nosso modelo? só saberemos daqui um tempo, mas devemos tentar


**DAQUI EM DIANTE, SEGUE UMA TENTAIVA FRACASSADA DE RECRIAR O DATAFRAME E ENCONTRAR UM OUTRO RESULTADO**

In [38]:
category_counts = playlists['new_category'].value_counts()
print(category_counts)

Theme                  14604
SingleArtist            6273
Rock/Pop                5404
AlternatingDJ           5053
IndieRock               4812
Romantic                2677
RoadTrip                1976
HipHop                  1517
Punk                    1338
Depression              1217
BreakUp                 1115
Narrative               1059
ElectronicMusic          858
Sleep                    746
Dance/House              706
Cover                    657
RhythmandBlues           496
Country                  484
Jazz                     466
Mixed                    424
Hardcore                 362
Indie                    357
Folk                     317
Ambient                  306
Experimental             298
WorldMusic               268
Reggae                   250
Rock                     227
Industrial               207
Blues                    197
Goth                     142
AvantGarde               110
Classical                 73
Drum&Bass                 58
Skate/Thrash  

In [39]:
max_count = category_counts.max()
max_count

14604

In [40]:
from sklearn.utils import resample




# Contar o número de exemplos por categoria
category_counts = playlists['new_category'].value_counts()

# Definir limites para oversampling e undersampling
min_count = 500
max_count = 500

# Balancear o dataset
balanced_playlists = pd.DataFrame()
for new_category, count in category_counts.items():
    label_df = playlists[playlists['new_category'] == new_category]
    if count < min_count:
        # Oversampling para categorias minoritárias
        label_df_resampled = resample(label_df, replace=True, n_samples=min_count, random_state=42)
    elif count > max_count:
        # Undersampling para categorias majoritárias
        label_df_resampled = resample(label_df, replace=False, n_samples=max_count, random_state=42)
    else:
        label_df_resampled = label_df
    balanced_playlists = pd.concat([balanced_playlists, label_df_resampled])

# Embaralhar o dataset balanceado
balanced_playlists = balanced_playlists.sample(frac=1, random_state=42).reset_index(drop=True)

# Verificar o resultado
balanced_playlists.groupby('new_category').count()


,playlists_limpas_indicadas,timestamp
new_category,,
AlternatingDJ,500,500
Ambient,500,500
AvantGarde,500,500
Blues,500,500
BreakUp,500,500
Classical,500,500
Country,500,500
Cover,500,500
Dance/House,500,500


In [41]:
train,test = train_test_split(balanced_playlists,test_size = 0.2)

In [42]:
display(train.shape)
display(test.shape)

(16800, 3)

(4200, 3)

In [43]:
train.to_csv(r'C:\\Caio\\Owl\\Train and test\\new_new_new_traine.csv',columns=['playlists_limpas_indicadas'],index=False,header=False)
test.to_csv(r'C:\\Caio\\Owl\\Train and test\\new_new_new_teste.csv',columns=['playlists_limpas_indicadas'],index=False,header=False)

In [44]:
modelo_final = fasttext.train_supervised(
    input=r"C:\\Caio\\Owl\\Train and test\\new_new_new_traine.csv")

In [45]:
modelo_final.test(r"C:\\Caio\\Owl\\Train and test\\new_new_new_teste.csv")

(4200, 0.39261904761904765, 0.39261904761904765)

In [46]:
modelo_final.predict('SnoppyDog')

(('__label__Ambient',), array([0.0355715]))